In [1]:
# Imports you may need
from dataloader import *
from finance import *
import pandas as pd

from finance import stock, compare
from quotebankexploration import *
from wikipedia import *

%load_ext autoreload
%autoreload 2

In [5]:
# Get some data frames
wiki_data = concat_wiki_files()
filtered_quotes = get_filtered_quotes()
speakers_id = pd.DataFrame(get_speakers_ids(filtered_quotes))
speakers_labels = get_speakers_labels_one_file()

In [10]:
quotes_test = filtered_quotes.head(10)
id_test = pd.DataFrame(get_speakers_ids(quotes_test))
labels_test = id_test.copy()
labels_test['label'] = find_labels(labels_test, wiki_data)

ValueError: Wrong number of items passed 3, placement implies 1

In [ ]:
display(quotes_test)

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2008-10-15-012147,five times the graphics performance.,steve jobs,[Q19837],2008-10-15 06:38:46,2,"[[steve jobs, 0.7707], [None, 0.2293]]",[http://us.rd.yahoo.com/dailynews/rss/search/m...,A
1,2008-09-25-038377,my iphone is full of pictures. i wanted to see...,ken whisenhunt,[Q1758635],2008-09-25 08:08:08,1,"[[ken whisenhunt, 0.3759], [None, 0.3186], [ma...",[http://azcentral.com/arizonarepublic/sports/a...,A
2,2008-09-04-052789,the people who connect needs and ideas the bes...,steve jobs,[Q19837],2008-09-04 18:19:47,1,"[[steve jobs, 0.717], [None, 0.283]]",[http://businessweek.com/magazine/content/07_4...,A
3,2008-11-10-038937,sesame workshop will get 70 percent of the rev...,robert macmillan,"[Q21453558, Q28094302]",2008-11-10 22:59:44,1,"[[robert macmillan, 0.6071], [None, 0.3929]]",[http://macdailynews.com/index.php/weblog/comm...,A
4,2008-09-26-048683,the huge success of the apple itunes app store...,steve howard,"[Q23670647, Q2457386, Q7612886]",2008-09-26 11:29:23,1,"[[steve howard, 0.5034], [None, 0.4966]]",[http://ipod.consumerelectronicsnet.com/articl...,A
5,2008-11-24-055069,we have a tough game against a tough opponent....,doug martin,"[Q18685889, Q27995830, Q3037945, Q3714598, Q53...",2008-11-24 21:48:30,1,"[[doug martin, 0.5937], [None, 0.4063]]",[http://kentstatesports.com//ViewArticle.dbml?...,A
6,2008-12-19-024329,it just takes the right kind of auto battery -...,steve jobs,[Q19837],2008-12-19 15:46:22,1,"[[steve jobs, 0.2931], [thomas friedman, 0.287...",[http://jpost.com/servlet/Satellite?cid=122872...,A
7,2008-09-30-047693,"the copyright royalty board in washington, d.c...",eddy cue,[Q224628],2008-09-30 20:18:30,1,"[[eddy cue, 0.5047], [None, 0.4953]]",[http://macdailynews.com/index.php/weblog/comm...,A
8,2008-12-17-065231,"whatever the reason, the unexpected announceme...",steve jobs,[Q19837],2008-12-17 15:33:00,2,"[[steve jobs, 0.5345], [None, 0.2782], [david ...",[http://cultofmac.com/oppenheimer-downgrade-wh...,A
9,2008-09-09-008132,"california has a spending problem, not a reven...",tom harman,[Q7816093],2008-09-09 15:24:34,1,"[[tom harman, 0.6994], [None, 0.3006]]",[http://iwuruss.wordpress.com/2008/09/09/itune...,A


In [ ]:
display(id_test)

,speaker,qids
0,steve jobs,[Q19837]
1,ken whisenhunt,[Q1758635]
2,robert macmillan,"[Q21453558, Q28094302]"
3,steve howard,"[Q23670647, Q2457386, Q7612886]"
4,doug martin,"[Q18685889, Q27995830, Q3037945, Q3714598, Q53..."
5,eddy cue,[Q224628]
6,tom harman,[Q7816093]


In [ ]:
display(labels_test)

,speaker,qids,label
0,steve jobs,[Q19837],Steve Jobs
1,ken whisenhunt,[Q1758635],Ken Whisenhunt
2,robert macmillan,"[Q21453558, Q28094302]",Robert MacMillan
3,steve howard,"[Q23670647, Q2457386, Q7612886]",Steve Howard
4,doug martin,"[Q18685889, Q27995830, Q3037945, Q3714598, Q53...",Doug Martin
...,...,...,...
40195,alex holmes,[Q4717192],Alex Holmes
40196,darnell nurse,[Q13486477],Darnell Nurse
40197,paul stone,"[Q26832781, Q56098687, Q57733391]",Paul Stone
40198,justin everett,[Q16210933],Justin Everett


In [ ]:
# test = speakers_labels.head(30)
for year in range(2015, 2021):
    path = os.path.join('data/wiki_speaker_attributes/speakers_pageviews_'+str(year))
    speakers_labels[str(year)] = speakers_labels.label.apply(lambda label: get_page_views_per_year(label, year))
    speakers_labels.to_pickle(path)
    print(year)

In [ ]:
display(speakers_labels.head(10))

In [ ]:
print(get_page_views_per_year('Shameel Joosub', 2015))

In [ ]:
yolo = filtered_quotes.copy()
yolo['year'] = filtered_quotes.date.apply(lambda date: date.year)
display(yolo)